In [ ]:
!pip install wit


  Preparing metadata (setup.py) ... done
  Created wheel for wit: filename=wit-6.0.1-py3-none-any.whl size=5296 sha256=2f63f194c2d961e7b3371921cb49ca379050a7229e5405cf0668d446d6851607
  Stored in directory: /root/.cache/pip/wheels/a9/ea/7d/55ba15bb7612a6adea29f8085ad022e657c0ec8ba6ec6e7fc4
Successfully built wit


In [ ]:
import requests
import pandas as pd
import os
import time
from tqdm import tqdm
import re

#  Wit.ai Server Access Token
wit_ai_token = 'AOH5LRVAGSVUQU5X2W2XNVLXGEKTUI2B'

# Audio files directory
audio_dir = "/content/drive/MyDrive/audio_files/Dataset_2_Cleaned"

# transcribe audio using Wit.ai
def transcribe_audio_wit(file_path):
    headers = {
        'Authorization': f'Bearer {wit_ai_token}',
        'Content-Type': 'audio/wav',
    }
    with open(file_path, 'rb') as audio_file:
        response = requests.post('https://api.wit.ai/speech?v=20200513', headers=headers, data=audio_file)
    if response.status_code == 200:
        data = response.json()
        print(f"Response for {file_path}: {data}")  # Detailed logging
        if 'text' in data:
            text = data['text'].upper().strip()
            return text, 1.0  #  use 1.0 as placeholder
    else:
        print(f"Error {response.status_code} for {file_path}: {response.text}")  # Error logging
        if response.status_code == 429:
            raise Exception("rate-limit")  # Raise an exception for rate limit errors
    return "", 0.0

# Clean and normalize expected words
def clean_expected_word(word):
    # Remove any numeric prefix and spaces
    word = re.sub(r'^[\d_]+\s*', '', word)
    return word.upper().strip()

# Get all wav files
wav_files = [os.path.join(root, file) for root, _, files in os.walk(audio_dir) for file in files if file.endswith(".wav")]

# Rate limiting parameters
requests_per_minute = 60
delay_between_requests = 60 / requests_per_minute
max_retries = 5

#  process files with retry logic and exponential backoff
def process_file(file_path):
    retries = 0
    while retries < max_retries:
        try:
            detected_word, confidence = transcribe_audio_wit(file_path)
            return detected_word, confidence
        except Exception as e:
            if str(e) == "rate-limit":
                retries += 1
                sleep_time = delay_between_requests * (2 ** retries)  # Exponential backoff
                print(f"Rate limit exceeded. Sleeping for {sleep_time} seconds.")
                time.sleep(sleep_time)
            else:
                print(f"Error: {e}")
                return "", 0.0
    return "", 0.0

# Process each wav file with rate limiting and retry logic
results = []
for i, file_path in enumerate(tqdm(wav_files, desc="Processing audio files")):
    detected_word, confidence = process_file(file_path)

    expected_word = os.path.basename(os.path.dirname(file_path)).upper().strip()
    # Clean the expected word
    expected_word = clean_expected_word(expected_word)

    correct = 1 if detected_word == expected_word else 0

    results.append({
        "File": file_path,
        "Expected Word": expected_word,
        "Detected Word": detected_word,
        "Confidence": confidence,
        "Correct": correct
    })

    # Apply rate limiting after each request
    time.sleep(delay_between_requests)

# Convert results to a DataFrame
df = pd.DataFrame(results)

# Save results to CSV file
df.to_csv("transcription_results_wit.csv", index=False)

# Print DataFrame
print(df)

print("Transcription results saved to transcription_results_wit.csv")


Processing audio files:   0%|          | 0/1848 [00:00<?, ?it/s]

Response for /content/drive/MyDrive/audio_files/Dataset_2_Cleaned/05_BOUQUET/P1.wav: {'entities': {}, 'intents': [], 'speech': {'confidence': 0.769, 'tokens': [{'end': 360, 'start': 0, 'token': 'Okay.'}]}, 'text': 'Okay', 'traits': {}}


Processing audio files:   0%|          | 1/1848 [00:01<54:52,  1.78s/it]

Response for /content/drive/MyDrive/audio_files/Dataset_2_Cleaned/05_BOUQUET/P2.wav: {'entities': {}, 'intents': [], 'speech': {'confidence': 0.8376, 'tokens': [{'end': 420, 'start': 0, 'token': 'Okay.'}]}, 'text': 'Okay', 'traits': {}}


Processing audio files:   0%|          | 2/1848 [00:03<50:49,  1.65s/it]

...

Response for /content/drive/MyDrive/audio_files/Dataset_2_Cleaned/08_DENY/P36.wav: {'entities': {}, 'intents': [], 'speech': {'confidence': 0.5047, 'tokens': [{'end': 720, 'start': 0, 'token': 'Die.'}]}, 'text': 'Die', 'traits': {}}


Processing audio files: 100%|█████████▉| 1847/1848 [56:44<00:01,  1.79s/it]

Response for /content/drive/MyDrive/audio_files/Dataset_2_Cleaned/08_DENY/P37.wav: {'entities': {}, 'intents': [], 'speech': {'confidence': 0.882, 'tokens': [{'end': 540, 'start': 0, 'token': 'Deny.'}]}, 'text': 'Deny', 'traits': {}}


Processing audio files: 100%|██████████| 1848/1848 [56:46<00:00,  1.84s/it]

                                                   File Expected Word  \
0     /content/drive/MyDrive/audio_files/Dataset_2_C...       BOUQUET   
1     /content/drive/MyDrive/audio_files/Dataset_2_C...       BOUQUET   
2     /content/drive/MyDrive/audio_files/Dataset_2_C...       BOUQUET   
3     /content/drive/MyDrive/audio_files/Dataset_2_C...       BOUQUET   
4     /content/drive/MyDrive/audio_files/Dataset_2_C...       BOUQUET   
...                                                 ...           ...   
1843  /content/drive/MyDrive/audio_files/Dataset_2_C...          DENY   
1844  /content/drive/MyDrive/audio_files/Dataset_2_C...          DENY   
1845  /content/drive/MyDrive/audio_files/Dataset_2_C...          DENY   
1846  /content/drive/MyDrive/audio_files/Dataset_2_C...          DENY   
1847  /content/drive/MyDrive/audio_files/Dataset_2_C...          DENY   

     Detected Word  Confidence  Correct  
0             OKAY         1.0        0  
1             OKAY         1.0        0